In [ ]:
# ================================================================
# 03_kerr_flux_sim.ipynb  –  Kerr-enhanced quaternionic flux (修正版)
# ================================================================
# Colab cell 1 : 基本套件 & 常數
!pip install --quiet scipy

import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from math import sqrt, pi

# ---- 物理常數 (SI) ------------------------------------------------
G     = 6.67430e-11          # m³ kg⁻¹ s⁻²
c     = 2.99792458e8         # m s⁻¹
M_sun = 1.98847e30           # kg
M_pl  = 2.176434e-8          # kg  (Planck 質量)

# ---- 黑洞參數（Sgr A* 示意） --------------------------------------
M_bh   = 4.0e6 * M_sun       # 4×10⁶ M☉
a_star = 0.5                 # 無因次自旋  a* = J/(M r_g)

r_g_m  = 2*G*M_bh / c**2     # Schwarzschild 半徑 (m)
print(f"Schwarzschild radius: {r_g_m/1e3:.2e} km,  a* = {a_star}")

# ---- 無因次化：r̂ = r / r_g，故 r_g→1，a_hat = a* ------------------
a_hat = a_star

# ---- Δ 與其導數 (無因次 r̂) ---------------------------------------
Δ  = lambda x: x**2 - x + a_hat**2
dΔ = lambda x: 2*x - 1

# ---- curvature-driven coupling λ̃(x) -----------------------------
kappa = 0.02
# 原函數 ∼ κ·(GM)/(Mpl²·r³)，轉成無因次後只剩 2κ/x³
λ_tilde = lambda x: 2*kappa / x**3

# ---- ODE：y=[ψ, χ], χ=dψ/dx --------------------------------------
def rhs(x, y):
    ψ, χ = y
    src = x**2 * λ_tilde(x) * ψ*(ψ**2 - 1)        # x²·λ̃·ψ(ψ²−1)
    dF = src - (2*x*Δ(x) + x**2*dΔ(x))*χ           # dF = source − d(x²Δ)·χ
    return [χ, dF / (x**2 * Δ(x))]

# ---- 積分範圍 r̂_+ → 100 ----------------------------------------
x_plus  = (1 + sqrt(1 - a_hat**2)) / 2
x_start = x_plus * 1.02
x_end   = 100.0
x_arr   = np.logspace(np.log10(x_start), np.log10(x_end), 600)

# ---- 求解 ODE（指定 t_eval） ------------------------------------
y0 = [1e-6, 0.0]   # ψ≈0, ψ'≈0 at horizon
sol = solve_ivp(rhs,
                t_span=(x_start, x_end),
                y0=y0,
                method='RK45',
                rtol=1e-9, atol=1e-12,
                t_eval=x_arr)

ψ_arr = sol.y[0]       # 無因次 ψ
ε_arr = 2*ψ_arr        # flux amplitude ε(r)=2ψ

# ---- Analytic ansatz 比較（論文 Eq. E.5） ----------------------
# 這裡用同樣的無因次化做法
r_s0_hat = 8.5e3 * 3.085677e16 / r_g_m   # 8.5 kpc → 無因次
boost    = 1.0 / (1 - a_hat/x_arr)
ε_ansatz = 2*np.tanh(x_arr / (r_s0_hat * boost))

# ---- 繪圖 ε(r) ----------------------------------------------------
plt.figure(figsize=(7,5))
plt.loglog(x_arr,   ε_arr,    lw=2, label="Numerical ε(r)")
plt.loglog(x_arr, ε_ansatz, '--', lw=2, label="Analytic ansatz")
plt.axvline(x_plus, color='k', ls=':', alpha=0.5)
plt.text(1.05, 0.3, r"$r_+$", transform=plt.gca().transAxes)
plt.xlabel(r"$\hat r = r/r_g$")
plt.ylabel(r"$\epsilon(r)$")
plt.title("Quaternionic flux in Kerr spacetime")
plt.legend(); plt.grid(ls='--', alpha=0.3)
plt.tight_layout(); plt.show()

# ---- δc_g/c 估算 (f=0.01Hz, LISA) -----------------------------
f_LISA = 1e-2     # Hz
k      = 2*pi*f_LISA / c   # s⁻¹ (c in m/s)
idx10  = np.argmin(abs(x_arr - 10))
ψ10    = ψ_arr[idx10]
deltac = (ψ10**2 * (a_hat/x_arr[idx10])**2) / (2 * k**2)

print(f"\nAt 10 r_g: ε ≈ {ε_arr[idx10]:.2e},  δc_g/c ≈ {deltac:.2e}")
